# **Lab 02: Khai thác và trực quan hóa dữ liệu**

### **Danh sách các thành viên**

<center>

|         Họ tên         |   MSSV   |
| :--------------------- | :------: |
| Trần Nguyễn Nhật Cường | 22127048 |
|    Nguyễn Công Tuấn    | 22127436 |
|     Trần Đăng Tuấn     | 22127438 |

</center>

## **Cài đặt và sử dụng các thư viện cần thiết**

In [1]:
%pip install matplotlib pandas seaborn kaggle scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import zipfile
import os
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

pd.options.display.float_format = "{:.2f}".format

**`matplotlib`** được dùng để vẽ biểu đồ trong Python. Có thể dùng để vẽ các loại biểu đồ như
histogram, biểu đồ tán xạ (scatter plot), biểu đồ đường (line chart) và biểu đồ cột (bar chart),...

**`pandas`** được dùng để xử lý và phân tích dữ liệu dạng bảng và cung cấp cấu trúc dữ liệu như
DataFrame và Series để dễ thao tác

**`seaborn`** dùng để vẽ biểu đồ dựa trên thư viện matplotlib nhưng biểu đồ sẽ trông đẹp hơn và cũng hỗ trợ các loại biểu đồ thống kê như violin plot, box plot, heatmap

**`sklearn (scikit-learn)`** là thư viện học máy phổ biến trong Python, được sử dụng để thực hiện các tác vụ học máy và xử lý dữ liệu. Trong đồ án này, các thành phần sau của sklearn được sử dụng:
- **`sklearn.cluster.KMeans`**: Cung cấp thuật toán KMeans để gom nhóm dữ liệu thành các cụm dựa trên sự tương đồng, được sử dụng để phân loại các nền kinh tế thành các nhóm.
- **`sklearn.metrics.silhouette_score`**: Dùng để tính điểm Silhouette, một chỉ số đánh giá chất lượng của việc gom nhóm, giúp xác định số lượng cụm tối ưu bằng cách đo lường mức độ tách biệt giữa các cụm.
- **`sklearn.preprocessing.StandardScaler`**: Dùng để chuẩn hóa dữ liệu, đưa các giá trị về cùng thang đo (trung bình bằng 0 và độ lệch chuẩn bằng 1), đảm bảo các chỉ số kinh tế có trọng số công bằng khi áp dụng thuật toán gom nhóm.

## **Thu thập dữ liệu**

*Nền tảng thu thập* - *Kaggle*

*Tên bộ dữ liệu thu thập* - Student Depression Dataset: Analyzing Mental Health Trends and Predictors Among Students

*Thông tin về bộ dữ liệu* - Bộ dữ liệu này được tổng hợp từ nhiều nguồn khác nhau nhằm mục đích phân tích và dự đoán mức độ trầm cảm ở sinh viên. Nó được thiết kế phục vụ nghiên cứu trong các lĩnh vực tâm lý học, khoa học dữ liệu và giáo dục. Bộ dữ liệu cung cấp những hiểu biết quan trọng về các yếu tố góp phần vào các vấn đề sức khỏe tâm thần của sinh viên, từ đó hỗ trợ xây dựng các chiến lược can thiệp sớm và cải thiện sức khỏe tinh thần cho đối tượng này.

In [14]:
!kaggle datasets download -d adilshamim8/student-depression-dataset -p ./

Dataset URL: https://www.kaggle.com/datasets/adilshamim8/student-depression-dataset
License(s): apache-2.0




  0%|          | 0.00/456k [00:00<?, ?B/s]
100%|██████████| 456k/456k [00:00<00:00, 593kB/s]
100%|██████████| 456k/456k [00:00<00:00, 593kB/s]


In [15]:
zip_file_path = './student-depression-dataset.zip'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall('./')

os.remove(zip_file_path) 
csv_file_path = os.path.join('./', 'student_depression_dataset.csv') 

## **Đọc bộ dữ liệu**

In [17]:
X = pd.read_csv(csv_file_path)

In [18]:
X.head()

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,2,Male,33.00,Visakhapatnam,Student,5.00,0.00,8.97,2.00,0.00,'5-6 hours',Healthy,B.Pharm,Yes,3.00,1.0,No,1
1,8,Female,24.00,Bangalore,Student,2.00,0.00,5.90,5.00,0.00,'5-6 hours',Moderate,BSc,No,3.00,2.0,Yes,0
2,26,Male,31.00,Srinagar,Student,3.00,0.00,7.03,5.00,0.00,'Less than 5 hours',Healthy,BA,No,9.00,1.0,Yes,0
3,30,Female,28.00,Varanasi,Student,3.00,0.00,5.59,2.00,0.00,'7-8 hours',Moderate,BCA,Yes,4.00,5.0,Yes,1
4,32,Female,25.00,Jaipur,Student,4.00,0.00,8.13,3.00,0.00,'5-6 hours',Moderate,M.Tech,Yes,1.00,1.0,No,0


## **Khám phá dữ liệu**

Bảng bên dưới mô tả chi tiết ý nghĩa của các thuộc tính trong bộ dữ liệu **`Student Depression Dataset`**


<center>

| **Tên thuộc tính**                        | **Mô tả** |
|-------------------------------------------|-------------------------------------------------------------|
| **id**                                    | Mã định danh của sinh viên. |
| **Gender**                                | Giới tính của sinh viên. Giá trị có thể là 'Male' (Nam) hoặc 'Female' (Nữ). |
| **Age**                                   | Tuổi của sinh viên. |
| **City**                                  | Thành phố nơi sinh viên sinh sống. |
| **Profession**                            | Nghề nghiệp của sinh viên. Giá trị có thể là 'Student' (Sinh viên). |
| **Academic Pressure**                     | Mức độ áp lực học tập của sinh viên (từ 1 đến 5, với 1 là ít và 5 là rất nhiều). |
| **Work Pressure**                         | Mức độ áp lực công việc của sinh viên (từ 1 đến 5, với 1 là ít và 5 là rất nhiều). |
| **CGPA**                                  | Điểm trung bình tích lũy (CGPA) của sinh viên. |
| **Study Satisfaction**                    | Mức độ hài lòng với việc học của sinh viên (từ 1 đến 5). |
| **Job Satisfaction**                      | Mức độ hài lòng với công việc của sinh viên (từ 1 đến 5). |
| **Sleep Duration**                        | Thời gian ngủ trung bình trong một ngày của sinh viên. Các giá trị có thể là 'Less than 5 hours', '5-6 hours', '7-8 hours', hoặc hơn. |
| **Dietary Habits**                        | Thói quen ăn uống của sinh viên. Các giá trị có thể là 'Healthy' (Lành mạnh) hoặc 'Moderate' (Hợp lý). |
| **Degree**                                | Loại bằng cấp của sinh viên, ví dụ: 'B.Pharm', 'BSc', 'M.Tech', v.v. |
| **Have you ever had suicidal thoughts?**  | Câu trả lời về việc sinh viên đã từng có suy nghĩ tự tử hay chưa. Giá trị có thể là 'Yes' hoặc 'No'. |
| **Work/Study Hours**                      | Số giờ làm việc hoặc học tập mỗi ngày của sinh viên. |
| **Financial Stress**                      | Mức độ căng thẳng tài chính của sinh viên (từ 1 đến 5, với 1 là ít và 5 là rất nhiều). |
| **Family History of Mental Illness**      | Liệu gia đình của sinh viên có tiền sử bệnh tâm lý hay không. Giá trị có thể là 'Yes' hoặc 'No'. |
| **Depression**                            | Mức độ trầm cảm của sinh viên. Giá trị có thể là '0' (không trầm cảm) hoặc '1' (trầm cảm). |


</center>



### **Tổng quan về dữ liệu**

In [19]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27901 entries, 0 to 27900
Data columns (total 18 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   id                                     27901 non-null  int64  
 1   Gender                                 27901 non-null  object 
 2   Age                                    27901 non-null  float64
 3   City                                   27901 non-null  object 
 4   Profession                             27901 non-null  object 
 5   Academic Pressure                      27901 non-null  float64
 6   Work Pressure                          27901 non-null  float64
 7   CGPA                                   27901 non-null  float64
 8   Study Satisfaction                     27901 non-null  float64
 9   Job Satisfaction                       27901 non-null  float64
 10  Sleep Duration                         27901 non-null  object 
 11  Di

**Nhận xét về bộ dữ liệu Nấm (tập feature X) sau khi xử lý**
- Dữ liệu còn *27901* dòng và *18* thuộc tính (cột).
- Kiểu dữ liệu bao gồm, int, float và object.
- Không có giá trị khuyết thiếu trong bất kỳ thuộc tính nào vì tất cả các cột đều có 27901 giá trị không null.

## **Các câu hỏi**

#### **Câu 1**: ...


**Thuộc tính được sử dụng**
1. ..
2. ..

**Các bước thực hiện**
1. 
2. 
3.
4. 
5.

**Nhận xét**

